In [2]:
import random
# N amount of robots, K amount corrupted

#Problem #1: If you are one of the uncorrupted robots, what strategy
# can you suggest for your colleagues (other uncorrupted robots)
# to find all K corrupt robots within the minimum number of usages of the test bed?


# Problem #2: If you are one of the corrupted robots, what strategy can you suggest for your
# colleagues (other corrupted robots) to avoid or
# delay detection as much as possible (and hence maximize the usages of the test bed)?

# Problem 3: What is the minimum number K such that it is impossible for any
# algorithm to definitively prove which robot is corrupted and which is not?


class TestBed():
    def __init__(self):
        self.spaces = {0: 0, 1: 0}
        self.visits = 0

    def resetSpaces(self):
        self.spaces = {0: 0, 1: 0}

    def placeRobot(self, spot, robot):
        self.spaces.update({spot: robot})

    def testSpots(self):
        self.visits += 1
        hunter = self.spaces[0]
        target = self.spaces[1]

        # target = self.spaces[1 if i < 1 else 0]
        # print('Target ',target.id)
        # print('Hunter ', hunter.id)

        # hunter checks second spot
        if hunter.corrupted == False and target.corrupted == True:
            target.corruptedAccusations += 1
            return target.corrupted

        elif hunter.corrupted == False and target.corrupted == False:
            target.nonCorruptedAccusations += 1
            return target.corrupted

        elif hunter.corrupted == True:
            choice = random.choice([True, False])
            if choice == True:
                target.corruptedAccusations += 1
            else:
                target.nonCorruptedAccusations += 1
        # target.tested = True

        # target checks first spot
        elif target.corrupted == False and hunter.corrupted == True:
            hunter.corruptedAccusations += 1
            return hunter.corrupted

        elif target.corrupted == False and hunter.corrupted == False:
            hunter.nonCorruptedAccusations += 1
            return hunter.corrupted

        elif target.corrupted == True:
            choice = random.choice([True, False])
            if choice == True:
                hunter.corruptedAccusations += 1
            else:
                hunter.nonCorruptedAccusations += 1
        target.tested = True
        hunter.tested = True


class Robot():
    def __init__(self, id, corrupted):
        self.corrupted = corrupted
        self.id = id
        self.tested = False
        self.terminated = False
        self.corruptedAccusations = 0
        self.nonCorruptedAccusations = 0
        self.difference = 0
        self.tester = False

    def createRobots(num):
        robots = []

        for i in range(num):
            if random.randint(0, num) < 0.5*num:
                state = False
            else:
                state = True
            robots.append(Robot(i, state)) # To be changed for random corruption
        return robots


def assess(robot):
    danger = robot.corruptedAccusations
    safe = robot.nonCorruptedAccusations
    robot.difference = ((danger-safe)/((danger+safe)))*100
    # print(robot.id, ' Dif: ', robot.difference)
    # print(robot.terminated, robot.tester)

    # Change numbers to adjust percentage required to deem robot either corrupted or not
    if robot.difference >= 60:
        robot.terminated = True

    if robot.difference <= -60:
        robot.tester = True

# Working progress, not sure yet

def testRobots(bed, robots):
    Run = False
    while Run == False:
        bossRobot = ''
        for i in robots:
            if i.tester == True:
                bossRobot = i

        if bossRobot != '':
            bed.placeRobot(0, bossRobot)
            for i in robots:
                if (i != bossRobot) and (i.terminated or i.tester != True):
                    bed.placeRobot(1, i)
                    bed.testSpots()
                    # assess(i)

        else:

            for i in robots:
                bed.placeRobot(0, i)

                for j in robots:
                    if (i != j) and (i.terminated or j.terminated != True):
                        bed.placeRobot(1, j)
                        bed.testSpots()
                        # assess(i)

        for i in robots:
            assess(i)

        Run = all(i.tester or i.terminated for i in robots)
        print(bed.visits)
        print(sum(robot.corrupted for robot in robots))

def printStates(robots):
    for i in robots:
        print('\nRobot ',i.id, '\n is Tested:     ', i.tested)
        print(' is Corrupted:  ', i.corrupted)
        print(' is Terminated: ', i.terminated)
        print(' is Tester:     ', i.tester)
        print(' Accusations:   ', i.corruptedAccusations)
        print(' SafeVote:      ', i.nonCorruptedAccusations)
        print(' Difference:    ', i.difference, '\n')


    print(bed.visits)
    print(sum(robot.corrupted for robot in robots))

In [3]:
# TESTING AREA
bed = TestBed()
robots = Robot.createRobots(20)

testRobots(bed, robots)
printStates(robots)

380
11
398
11
415
11
432
11
449
11
466
11
483
11
499
11
515
11
531
11
547
11
563
11
577
11
591
11
605
11
619
11
633
11

Robot  0 
 is Tested:      True
 is Corrupted:   True
 is Terminated:  True
 is Tester:      False
 Accusations:    30
 SafeVote:       5
 Difference:     71.42857142857143 


Robot  1 
 is Tested:      True
 is Corrupted:   False
 is Terminated:  False
 is Tester:      True
 Accusations:    6
 SafeVote:       24
 Difference:     -60.0 


Robot  2 
 is Tested:      True
 is Corrupted:   True
 is Terminated:  True
 is Tester:      False
 Accusations:    29
 SafeVote:       6
 Difference:     65.71428571428571 


Robot  3 
 is Tested:      True
 is Corrupted:   True
 is Terminated:  True
 is Tester:      False
 Accusations:    29
 SafeVote:       6
 Difference:     65.71428571428571 


Robot  4 
 is Tested:      True
 is Corrupted:   False
 is Terminated:  False
 is Tester:      True
 Accusations:    3
 SafeVote:       16
 Difference:     -68.42105263157895 


Robot  5 

This is an example run with 11 corrupted robots (amount is randomly generated), The tactic in this current program for the corrupted robots is just to answer randomly while the non corrupted robots chooses a "Tester" as soon as one is determined (Done with difference percentages between accusations and "safe votes"). Usually it runs between 500 - 1000 visits (20 robots) if they can determine a tester, but if the corrupted robots are lucky and votes often enough that the other one is corrupted then the visit count rises to alot of digits. Example: 7434496 visits with 10 corrupted robots, it has gotten it done with 13 corrupted so this example is based on the random in voting for corrupted robots. At least on my computer it gets done in maximum 1 sec (runs it locally) I would say that the instant it goes over 1sec I kinda know it won't finnish.

Strategy for non corrupted is as I explained and that what this code already uses. To choose one robot as a tester when it becomes safe to determine it as a non corrupted robot

Strategy for corrupted robots: Just wote Corrupt on every non corrupted robot (not sure in this task what does the robots actually know, for example do they know who the other robot on the test bed is?) This program runs it as such that the corrupted robots votes randomly even if the other one is corrupted as well.

The highest amount of corrupted robots (with 20 robots) that it solved was 12, I spam ran the program to test it, sometimes it failed at even 9. In conclusion > 12 would be the  limit for this one at least. In theory it could be done with more but the odds are quite small.